In [ ]:
write = True # log this version as new
MODEL_DESCRIPTION = "Diffusion"

In [ ]:
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
from PIL import Image

import tensorflow as tf;
from tensorflow import keras, einsum
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Layer
import tensorflow.keras.layers as nn
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import ReduceLROnPlateau

from einops import rearrange
from einops.layers.tensorflow import Rearrange
from functools import partial
from inspect import isfunction

import pandas as pd
from sys import stdout
from datetime import datetime
import neptune as neptune
from neptune.integrations.tensorflow_keras import NeptuneCallback
from astropy.io import fits
import random
from DataMaker import HDF5ImageGenerator

In [ ]:
%%capture
gpus = tf.config.list_physical_devices('GPU');
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit = 20000)]);
        logical_gpus = tf.config.experimental.list_logical_devices('GPU');
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

In [ ]:
now = datetime.now()

DATASET_NAME = "HSC_v6"
MODEL_TYPE = "Diffusion"
MODEL_VERSION = "v1.0"
MODEL_SUBVERSION = "v1.0." + now.strftime("%Y%m%d_%H_%M_%S")

model_id = '_'.join([DATASET_NAME, MODEL_TYPE, MODEL_VERSION])
model_path = os.path.join('/data3/Billy/models', model_id, MODEL_SUBVERSION, 'model')
checkpoints_path = os.path.join('/data3/Billy/models', model_id, MODEL_SUBVERSION, 'checkpoints')
logs_path = os.path.join('/data3/Billy/logs', model_id, MODEL_SUBVERSION)
predictions_path = os.path.join('/data3/Billy/predictions', model_id, MODEL_SUBVERSION)
weights_path = model_path + '/data3/Billy/Diffusion/weights.h5'

os.makedirs(model_path, exist_ok = True)
os.makedirs(checkpoints_path, exist_ok = True)
os.makedirs(logs_path, exist_ok = True)
os.makedirs(predictions_path, exist_ok = True)

if write == True:
    with open("/data3/Billy/models/README.md", "a") as myfile:
        myfile.write(MODEL_TYPE + " " + MODEL_SUBVERSION + " - " + MODEL_DESCRIPTION + " - B. Li" + "\n")
else:
    with open("/data3/Billy/models/README.md", "a") as myfile:
        myfile.write(MODEL_TYPE + " " + MODEL_SUBVERSION + " - ... "+ " - B. Li" + "\n")

In [ ]:
channels = 5
BATCH_SIZE = 8

hf_train = h5py.File('/data/HSC/HSC_v6/step2A/127x127/5x127x127_training_small.hdf5', 'r')
hf_test = h5py.File('/data/HSC/HSC_v6/step2A/127x127/5x127x127_testing_small.hdf5', 'r')
hf_validation = h5py.File('/data/HSC/HSC_v6/step2A/127x127/5x127x127_validation_small.hdf5', 'r')
x_train = np.asarray(np.transpose(hf_train['image'][0:],(0,2,3,1)))
x_test = np.asarray(np.transpose(hf_test['image'][0:],(0,2,3,1)))
x_validation = np.asarray(np.transpose(hf_validation['image'][0:],(0,2,3,1)))
x_train = np.pad(x_train, [(0, 0), (0, 1), (0, 1), (0, 0)], mode = 'constant') # Padding to 128x128
x_test = np.pad(x_test, [(0, 0), (0, 1), (0, 1), (0, 0)], mode = 'constant') # Padding to 128x128
x_validation = np.pad(x_validation, [(0, 0), (0, 1), (0, 1), (0, 0)], mode = 'constant') # Padding to 128x128
y_train = np.asarray(hf_train['specz_redshift'][0:])[..., None]
y_test = np.asarray(hf_test['specz_redshift'][0:])[..., None]
y_validation = np.asarray(hf_validation['specz_redshift'][0:])[..., None]
hf_train.close()
hf_test.close()
hf_validation.close()

In [ ]:
batch_count = 0
def next_batch():
    global batch_count
    hf_train = h5py.File('/data/HSC/HSC_v6/step2A/127x127/5x127x127_training_small.hdf5', 'r')
    
    x_train = np.asarray(np.transpose(hf_train['image'][0:],(0,2,3,1)))
    for i in range(len(x_train)):
        x_train[i] = np.true_divide(x_train[i], np.max(np.nan_to_num(x_train[i])))
    x_train = x_train[batch_count * BATCH_SIZE : batch_count * BATCH_SIZE + BATCH_SIZE]
    x_train = np.pad(x_train, [(0, 0), (0, 1), (0, 1), (0, 0)], mode = 'constant') # Padding to 128x128
    hf_train.close()
    batch_count = batch_count + 1
    if batch_count == 10000 / BATCH_SIZE - 1:
        batch_count = 0
    return x_train

In [ ]:
timesteps = 200
max_noise = 0.1

# create a fixed beta schedule
beta = np.linspace(max_noise / timesteps, max_noise, timesteps)

# this will be used as discussed in the reparameterization trick
alpha = 1 - beta
alpha_bar = np.cumprod(alpha, 0)
alpha_bar = np.concatenate((np.array([1.]), alpha_bar[ : -1]), axis = 0)
sqrt_alpha_bar = np.sqrt(alpha_bar)
one_minus_sqrt_alpha_bar = np.sqrt(1 - alpha_bar)

# this function will help us set the RNG key for Numpy
def set_key(key):
    np.random.seed(key)

# this function will add noise to the input as per the given timestamp
def forward_noise(key, x_0, t):
    set_key(key)
    noise = np.random.normal(size = x_0.shape)
    reshaped_sqrt_alpha_bar_t = np.reshape(np.take(sqrt_alpha_bar, t), (-1, 1, 1, 1))
    reshaped_one_minus_sqrt_alpha_bar_t = np.reshape(np.take(one_minus_sqrt_alpha_bar, t), (-1, 1, 1, 1))
    noisy_image = reshaped_sqrt_alpha_bar_t  * x_0 + reshaped_one_minus_sqrt_alpha_bar_t  * noise
    return noisy_image, noise

# this function will be used to create sample timestamps between 0 & T
def generate_timestamp(key, num):
    set_key(key)
    return tf.random.uniform(shape = [num], minval = 0, maxval = timesteps, dtype = tf.int32)

In [ ]:
# Let us visualize the output image at a few timestamps
sample = next_batch()[0]

In [ ]:
y_arr = np.array([])
for index, i in enumerate(range(timesteps)):
    noisy_im, noise = forward_noise(0, np.expand_dims(sample, 0), np.array([i, ]))
    y_arr = np.append(y_arr, np.var(noisy_im[0][:, :, 0].flatten()))

x_arr = np.linspace(0, timesteps, timesteps)


# plt.plot(x_arr, y_arr)
# plt.title("")
# plt.show()

fig, ax1 = plt.subplots()
ax1.plot(x_arr, y_arr)
ax1.set_xlabel("Steps of Noise")
ax1.set_ylabel("Variance in Noised Image")

ax2 = ax1.twiny()
ax2.plot(beta, 0.5 * np.ones(timesteps), alpha = 0) # Create a dummy plot
ax2.set_xlabel("Variance of the Gaussian Noise Added at Each Step")
# fig.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows = 5, ncols = 5, figsize = (50, 50))

j = 0
for index, i in enumerate([0, 20, 100, 150, 195]):
    noisy_im, noise = forward_noise(0, np.expand_dims(sample, 0), np.array([i, ]))
    axes[0][j].imshow(noisy_im[0][:, :, 0], cmap = 'gray')
    axes[1][j].imshow(noisy_im[0][:, :, 1], cmap = 'gray')
    axes[2][j].imshow(noisy_im[0][:, :, 2], cmap = 'gray')
    axes[3][j].imshow(noisy_im[0][:, :, 3], cmap = 'gray')
    axes[4][j].imshow(noisy_im[0][:, :, 4], cmap = 'gray')
    j += 1
    

plt.show()

In [ ]:
# helpers functions
def exists(x):
    return x is not None

def default(val, d):
    if exists(val):
        return val
    return d() if isfunction(d) else d

# We will use this to convert timestamps to time encodings
class SinusoidalPosEmb(Layer):
    def __init__(self, dim, max_positions = 10000):
        super(SinusoidalPosEmb, self).__init__()
        self.dim = dim
        self.max_positions = max_positions

    def call(self, x, training = True):
        x = tf.cast(x, tf.float32)
        half_dim = self.dim // 2
        emb = math.log(self.max_positions) / (half_dim - 1)
        emb = tf.exp(tf.range(half_dim, dtype = tf.float32) * -emb)
        emb = x[:, None] * emb[None, :]

        emb = tf.concat([tf.sin(emb), tf.cos(emb)], axis = -1)

        return emb
        
# small helper modules
class Identity(Layer):
    def __init__(self):
        super(Identity, self).__init__()

    def call(self, x, training = True):
        return tf.identity(x)


class Residual(Layer):
    def __init__(self, fn):
        super(Residual, self).__init__()
        self.fn = fn

    def call(self, x, training = True):
        return self.fn(x, training = training) + x

def Upsample(dim):
    return nn.Conv2DTranspose(filters = dim, kernel_size = 4, strides = 2, padding = 'SAME')

def Downsample(dim):
    return nn.Conv2D(filters = dim, kernel_size = 4, strides = 2, padding = 'SAME')

class LayerNorm(Layer):
    def __init__(self, dim, eps = 1e-5, **kwargs):
        super(LayerNorm, self).__init__(**kwargs)
        self.eps = eps

        self.g = tf.Variable(tf.ones([1, 1, 1, dim]))
        self.b = tf.Variable(tf.zeros([1, 1, 1, dim]))

    def call(self, x, training = True):
        var = tf.math.reduce_variance(x, axis = -1, keepdims = True)
        mean = tf.reduce_mean(x, axis = -1, keepdims = True)

        x = (x - mean) / tf.sqrt((var + self.eps)) * self.g + self.b
        return x

class PreNorm(Layer):
    def __init__(self, dim, fn):
        super(PreNorm, self).__init__()
        self.fn = fn
        self.norm = LayerNorm(dim)

    def call(self, x, training = True):
        x = self.norm(x)
        return self.fn(x)

class SiLU(Layer):
    def __init__(self):
        super(SiLU, self).__init__()

    def call(self, x, training = True):
        return x * tf.nn.sigmoid(x)

def gelu(x, approximate = False):
    if approximate:
        coeff = tf.cast(0.044715, x.dtype)
        return 0.5 * x * (1.0 + tf.tanh(0.7978845608028654 * (x + coeff * tf.pow(x, 3))))
    else:
        return 0.5 * x * (1.0 + tf.math.erf(x / tf.cast(1.4142135623730951, x.dtype)))

class GELU(Layer):
    def __init__(self, approximate = False):
        super(GELU, self).__init__()
        self.approximate = approximate

    def call(self, x, training = True):
        return gelu(x, self.approximate)

In [ ]:
# building block modules
class Block(Layer):
    def __init__(self, dim, groups = 8):
        super(Block, self).__init__()
        self.proj = nn.Conv2D(dim, kernel_size = 3, strides = 1, padding = 'SAME')
        self.norm = tfa.layers.GroupNormalization(groups, epsilon = 1e-05)
        self.act = SiLU()


    def call(self, x, gamma_beta = None, training = True):
        x = self.proj(x)
        x = self.norm(x, training=training)

        if exists(gamma_beta):
            gamma, beta = gamma_beta
            x = x * (gamma + 1) + beta

        x = self.act(x)
        return x

class ResnetBlock(Layer):
    def __init__(self, dim, dim_out, time_emb_dim = None, groups = 8):
        super(ResnetBlock, self).__init__()

        self.mlp = Sequential([
            SiLU(),
            nn.Dense(units = dim_out * 2)
        ]) if exists(time_emb_dim) else None

        self.block1 = Block(dim_out, groups = groups)
        self.block2 = Block(dim_out, groups = groups)
        self.res_conv = nn.Conv2D(filters = dim_out, kernel_size = 1, strides = 1) if dim != dim_out else Identity()

    def call(self, x, time_emb = None, training = True):
        gamma_beta = None
        if exists(self.mlp) and exists(time_emb):
            time_emb = self.mlp(time_emb)
            time_emb = rearrange(time_emb, 'b c -> b 1 1 c')
            gamma_beta = tf.split(time_emb, num_or_size_splits = 2, axis = -1)

        h = self.block1(x, gamma_beta = gamma_beta, training = training)
        h = self.block2(h, training = training)

        return h + self.res_conv(x)

class LinearAttention(Layer):
    def __init__(self, dim, heads = 4, dim_head = 127):
        super(LinearAttention, self).__init__()
        self.scale = dim_head ** -0.5
        self.heads = heads
        self.hidden_dim = dim_head * heads

        self.attend = nn.Softmax()
        self.to_qkv = nn.Conv2D(filters = self.hidden_dim * 3, kernel_size = 1, strides = 1, use_bias = False)

        self.to_out = Sequential([
            nn.Conv2D(filters = dim, kernel_size = 1, strides = 1),
            LayerNorm(dim)
        ])

    def call(self, x, training = True):
        b, h, w, c = x.shape
        qkv = self.to_qkv(x)
        qkv = tf.split(qkv, num_or_size_splits = 3, axis = -1)
        q, k, v = map(lambda t: rearrange(t, 'b x y (h c) -> b h c (x y)', h = self.heads), qkv)

        q = tf.nn.softmax(q, axis = -2)
        k = tf.nn.softmax(k, axis = -1)

        q = q * self.scale
        context = einsum('b h d n, b h e n -> b h d e', k, v)

        out = einsum('b h d e, b h d n -> b h e n', context, q)
        out = rearrange(out, 'b h c (x y) -> b x y (h c)', h = self.heads, x = h, y = w)
        out = self.to_out(out, training = training)

        return out

class Attention(Layer):
    def __init__(self, dim, heads = 4, dim_head = 127):
        super(Attention, self).__init__()
        self.scale = dim_head ** -0.5
        self.heads = heads
        self.hidden_dim = dim_head * heads

        self.to_qkv = nn.Conv2D(filters = self.hidden_dim * 3, kernel_size = 1, strides = 1, use_bias = False)
        self.to_out = nn.Conv2D(filters = dim, kernel_size = 1, strides = 1)

    def call(self, x, training = True):
        b, h, w, c = x.shape
        qkv = self.to_qkv(x)
        qkv = tf.split(qkv, num_or_size_splits = 3, axis = -1)
        q, k, v = map(lambda t: rearrange(t, 'b x y (h c) -> b h c (x y)', h=self.heads), qkv)
        q = q * self.scale

        sim = einsum('b h d i, b h d j -> b h i j', q, k)
        sim_max = tf.stop_gradient(tf.expand_dims(tf.argmax(sim, axis = -1), axis = -1))
        sim_max = tf.cast(sim_max, tf.float32)
        sim = sim - sim_max
        attn = tf.nn.softmax(sim, axis = -1)

        out = einsum('b h i j, b h d j -> b h i d', attn, v)
        out = rearrange(out, 'b h (x y) d -> b x y (h d)', x = h, y = w)
        out = self.to_out(out, training = training)

        return out

In [ ]:
class Unet(Model):
    def __init__(self,
                 dim = 128,
                 init_dim = None,
                 out_dim = None,
                 dim_mults = (1, 2, 4, 8),
                 channels = 5,
                 resnet_block_groups = 4,
                 learned_variance = False,
                 sinusoidal_cond_mlp = True
                 ):
        super(Unet, self).__init__()
        
        # determine dimensions
        self.channels = channels
        
        init_dim = default(init_dim, dim // 3 * 2)
        self.init_conv = nn.Conv2D(filters = init_dim, kernel_size = 7, strides = 1, padding = 'SAME')
        
        dims = [init_dim, *map(lambda m: dim * m, dim_mults)]
        in_out = list(zip(dims[ : -1], dims[1 : ]))
        
        block_klass = partial(ResnetBlock, groups = resnet_block_groups)
        
        # time embeddings
        time_dim = dim * 4
        self.sinusoidal_cond_mlp = sinusoidal_cond_mlp
        
        self.time_mlp = Sequential([
            SinusoidalPosEmb(dim),
            nn.Dense(units = time_dim),
            GELU(),
            nn.Dense(units = time_dim)
        ], name="time embeddings")
        
        # layers
        self.downs = []
        self.ups = []
        num_resolutions = len(in_out)
        
        for ind, (dim_in, dim_out) in enumerate(in_out):
            is_last = ind >= (num_resolutions - 1)

            self.downs.append([
                block_klass(dim_in, dim_out, time_emb_dim = time_dim),
                block_klass(dim_out, dim_out, time_emb_dim = time_dim),
                Residual(PreNorm(dim_out, LinearAttention(dim_out))),
                Downsample(dim_out) if not is_last else Identity()
            ])
  
        mid_dim = dims[-1]
        self.mid_block1 = block_klass(mid_dim, mid_dim, time_emb_dim = time_dim)
        self.mid_attn = Residual(PreNorm(mid_dim, Attention(mid_dim)))
        self.mid_block2 = block_klass(mid_dim, mid_dim, time_emb_dim = time_dim)
        
        for ind, (dim_in, dim_out) in enumerate(reversed(in_out[1:])):
            is_last = ind >= (num_resolutions - 1)

            self.ups.append([
                block_klass(dim_out * 2, dim_in, time_emb_dim = time_dim),
                block_klass(dim_in, dim_in, time_emb_dim = time_dim),
                Residual(PreNorm(dim_in, LinearAttention(dim_in))),
                Upsample(dim_in) if not is_last else Identity()
            ])
        
        default_out_dim = channels * (1 if not learned_variance else 2)
        self.out_dim = default(out_dim, default_out_dim)
        
        self.final_conv = Sequential([
            block_klass(dim * 2, dim),
            nn.Conv2D(filters = self.out_dim, kernel_size = 1, strides = 1)
        ], name = "output")
        
    def call(self, x, time = None, training = True, **kwargs):
        x = self.init_conv(x)
        t = self.time_mlp(time)
        
        h = []

        for block1, block2, attn, downsample in self.downs:
            x = block1(x, t)
            x = block2(x, t)
            x = attn(x)
            h.append(x)
            x = downsample(x)

        x = self.mid_block1(x, t)
        x = self.mid_attn(x)
        x = self.mid_block2(x, t)

        for block1, block2, attn, upsample in self.ups:
            x = tf.concat([x, h.pop()], axis = -1)
            x = block1(x, t)
            x = block2(x, t)
            x = attn(x)
            x = upsample(x)

        x = tf.concat([x, h.pop()], axis = -1)
        x = self.final_conv(x)
        return x

In [ ]:
# create our unet model
unet = Unet(channels = 5)

# initialize the model in the memory of our GPU
test_images = np.ones([1, 128, 128, channels])
test_timestamps = generate_timestamp(0, 1)
k = unet(test_images, test_timestamps)

# create our optimizer, we will use adam with a Learning rate of 1e-4
opt = keras.optimizers.Adam(learning_rate = 1e-4)

In [ ]:
unet.summary()

def loss_fn(real, generated):
    mse = tf.reshape((real - generated) ** 2, (channels * BATCH_SIZE * 128 * 128))
    real_flatten = tf.reshape(real, (channels * BATCH_SIZE * 128 * 128))

    mse = tf.cast(mse, dtype = tf.float32)
    real_flatten = tf.cast(tf.abs(real_flatten), dtype = tf.float32)

    loss_modified = tf.reduce_sum(tf.multiply(mse, real_flatten))
    return loss_modified

In [ ]:
def loss_fn(real, generated):
    loss = tf.math.reduce_mean((real - generated) ** 2)
    return loss

In [ ]:
rng = 0

def train_step(batch):
    rng, tsrng = np.random.randint(0, 100000, size = (2,))
    timestep_values = generate_timestamp(tsrng, batch.shape[0])

    noised_image, noise = forward_noise(rng, batch, timestep_values)
    with tf.GradientTape() as tape:
        prediction = unet(noised_image, timestep_values)
        loss_value = loss_fn(noise, prediction)
    
    gradients = tape.gradient(loss_value, unet.trainable_variables)
    opt.apply_gradients(zip(gradients, unet.trainable_variables))

    return loss_value

In [ ]:
epochs = 10
for e in range(1, epochs + 1):
    # this is cool utility in Tensorflow that will create a nice looking progress bar
    bar = tf.keras.utils.Progbar(10000 / 8 - 1)
    losses = []
    for i in range(0, int(10000 / 8)):
        batch = next_batch()
        loss = train_step(batch)
        losses.append(loss)
        bar.update(i, values = [("loss", loss)])
        
    avg = np.mean(losses)
    print(f"Average loss for epoch {e}/{epochs}: {avg}")

In [ ]:
stop here

In [ ]:
# Save a GIF using logged images

def save_gif(img_list, path="", interval=200):
    # Transform images from [-1,1] to [0, 255]
    imgs = []
    for im in img_list:
        im = np.array(im)
        im = (im + 1) * 127.5
        im = np.clip(im, 0, 255).astype(np.int32)
        im = Image.fromarray(im)
        imgs.append(im)
    
    imgs = iter(imgs)

    # Extract first image from iterator
    img = next(imgs)

    # Append the other images and save as GIF
    img.save(fp=path, format='GIF', append_images=imgs,
             save_all=True, duration=interval, loop=0)

In [ ]:
def ddpm(x_t, pred_noise, t):
    alpha_t = np.take(alpha, t)
    alpha_t_bar = np.take(alpha_bar, t)

    eps_coef = (1 - alpha_t) / (1 - alpha_t_bar) ** .5
    mean = (1 / (alpha_t ** .5)) * (x_t - eps_coef * pred_noise)

    var = np.take(beta, t)
    z = np.random.normal(size=x_t.shape)

    return mean + (var ** .5) * z

In [ ]:
for k in range(0, 5):
    for j in range(0, 5):
        x = tf.random.normal((1, 128, 128, 5))

        img_list = []
        img_list.append(np.squeeze(x, 0)[: , : , j])
    
        for i in tqdm(range(timesteps - 1)):
            t = np.expand_dims(np.array(timesteps - i - 1, np.int32), 0)
            pred_noise = unet(x, t)
            x = ddpm(x, pred_noise, t)
            img_list.append(np.squeeze(x, 0)[: , : , j])

        save_gif(img_list + ([img_list[-1]] * 100), f"diffusion_generations/rand_max_{max_noise}_image_{k}_band_{j}.gif", interval=20)

In [ ]:
stop here

In [ ]:
batch_count = 0
hf_train = h5py.File('/data/HSC/HSC_v6/step2A/127x127/5x127x127_testing_small.hdf5', 'r')
x_train = np.asarray(np.transpose(hf_train['image'][0:],(0,2,3,1)))
for i in range(len(x_train)):
    x_train[i] = np.true_divide(x_train[i], np.max(np.nan_to_num(x_train[i])))
x_train = np.pad(x_train, [(0, 0), (0, 1), (0, 1), (0, 0)], mode = 'constant') # Padding to 128x128
hf_train.close()

In [ ]:
sample = x_train[15]

for j in range(0, 5):
    x, noise_x = forward_noise(0, np.expand_dims(sample, 0), np.array([199, ]))
    print(x.shape)
    img_list = []
    img_list.append(np.squeeze(x, 0)[: , : , j])

    for i in tqdm(range(timesteps - 1)):
        t = np.expand_dims(np.array(timesteps - i - 1, np.int32), 0)
        pred_noise = unet(x, t)
        x = ddpm(x, pred_noise, t)
        img_list.append(np.squeeze(x, 0)[: , : , j])

    save_gif(img_list + ([img_list[-1]] * 100), f"diffusion_generations/five_band_{j}.gif", interval=20)

In [ ]:
for k in range(0, 10):
    sample = x_train[k]

    for j in range(0, 5):
        x, noise_x = forward_noise(0, np.expand_dims(sample, 0), np.array([199, ]))
        print(x.shape)
        img_list = []
        img_list.append(np.squeeze(x, 0)[: , : , j])

        for i in tqdm(range(timesteps - 1)):
            t = np.expand_dims(np.array(timesteps - i - 1, np.int32), 0)
            pred_noise = unet(x, t)
            x = ddpm(x, pred_noise, t)
            img_list.append(np.squeeze(x, 0)[: , : , j])

        save_gif(img_list + ([img_list[-1]] * 100), f"diffusion_generations/five_band_#{k}_band_{j}.gif", interval=20)

In [ ]:
for j in range(0, 100):

    sample = x_train[j]

    x, noise_x = forward_noise(0, np.expand_dims(sample, 0), np.array([199, ]))
    print(x.shape)
    img_list = []
    img_list.append(np.squeeze(x, 0)[: , : , 0])

    for i in tqdm(range(timesteps - 1)):
        t = np.expand_dims(np.array(timesteps - i - 1, np.int32), 0)
        pred_noise = unet(x, t)
        x = ddpm(x, pred_noise, t)
        img_list.append(np.squeeze(x, 0)[: , : , 0])

    save_gif(img_list + ([img_list[-1]] * 100), f"diffusion_generations/train_noise_{j}.gif", interval=20)

In [ ]:
sample = x_train[0]

print(sample.shape)

x, noise_x = forward_noise(0, np.expand_dims(sample, 0), np.array([timesteps - 1, ]))
print(x.shape)
img_list = []
img_list.append(np.squeeze(x, 0)[: , : , 0])

for i in tqdm(range(timesteps - 1)):
    t = np.expand_dims(np.array(timesteps - i - 1, np.int32), 0)
    pred_noise = unet(x, t)
    x = ddpm(x, pred_noise, t)

x = x[:, :-1, :-1, :] 
x = np.asarray(np.transpose(x,(0,3,1,2)))
print(x.shape)
plt.imshow(x[0][0])

In [ ]:
image_list_g = sorted(os.listdir("/data/HSC/HSC_v6/step1/g_band"))
image_list_r = sorted(os.listdir("/data/HSC/HSC_v6/step1/r_band"))
image_list_i = sorted(os.listdir("/data/HSC/HSC_v6/step1/i_band"))
image_list_z = sorted(os.listdir("/data/HSC/HSC_v6/step1/z_band"))
image_list_y = sorted(os.listdir("/data/HSC/HSC_v6/step1/y_band"))

photozdata = pd.read_csv('/data/HSC/HSC_v6/HSC_v6.csv')
photozdata.describe()

b = np.argsort(photozdata['object_id'])
sorted_photozdata = photozdata.iloc[b][:]
photozdata = sorted_photozdata

hf = h5py.File('/data3/Diffusion/127x127_diffusion_regenerated_normalized.hdf5', 'a')

In [ ]:
count_in = 0
count_out = 0
size = len(image_list_g)
for i in range(2000):
    
    stdout.write("\rChecking %d samples of " % (i + 1) + str(size))
    
    object_id = image_list_g[i][0:17]

    five_band_image = []

    image_g = fits.open("/data/HSC/HSC_v6/step1/g_band/" + image_list_g[i])
    image_r = fits.open("/data/HSC/HSC_v6/step1/r_band/" + image_list_r[i])
    image_i = fits.open("/data/HSC/HSC_v6/step1/i_band/" + image_list_i[i])
    image_z = fits.open("/data/HSC/HSC_v6/step1/z_band/" + image_list_z[i])
    image_y = fits.open("/data/HSC/HSC_v6/step1/y_band/" + image_list_y[i])

    image_g_data = image_g[1].data
    image_r_data = image_r[1].data
    image_i_data = image_i[1].data
    image_z_data = image_z[1].data
    image_y_data = image_y[1].data

    pad1 = int((127 - len(image_g_data)) / 2)
    pad2 = 127 - len(image_g_data) - pad1
    pad3 = int((127 - len(image_g_data[0])) / 2)
    pad4 = 127 - len(image_g_data[0]) - pad3


    im_g = np.pad(image_g_data, ((pad1, pad2), (pad3, pad4)), "constant", constant_values = ((0, 0), (0, 0)))
    im_r = np.pad(image_r_data, ((pad1, pad2), (pad3, pad4)), "constant", constant_values = ((0, 0), (0, 0)))
    im_i = np.pad(image_i_data, ((pad1, pad2), (pad3, pad4)), "constant", constant_values = ((0, 0), (0, 0)))
    im_z = np.pad(image_z_data, ((pad1, pad2), (pad3, pad4)), "constant", constant_values = ((0, 0), (0, 0)))
    im_y = np.pad(image_y_data, ((pad1, pad2), (pad3, pad4)), "constant", constant_values = ((0, 0), (0, 0)))

    im = np.true_divide(np.array([im_g, im_r, im_i, im_z, im_y]), np.max(np.array([im_g, im_r, im_i, im_z, im_y])))
    im = np.asarray(np.transpose(im, (1, 2, 0)))
    im = np.pad(im, [(0, 1), (0, 1), (0, 0)], mode = 'constant')
    x, noise_x = forward_noise(0, np.expand_dims(im, 0), np.array([timesteps - 1, ]))
    for k in tqdm(range(timesteps - 1)):
        t = np.expand_dims(np.array(timesteps - k - 1, np.int32), 0)
        pred_noise = unet(x, t)
        x = ddpm(x, pred_noise, t)
    
    x = x[:, :-1, :-1, :] 
    five_band_image_reshape = np.asarray(np.transpose(x, (0, 3, 1, 2)))
    
    if True:
    
        photozdata_subset = photozdata.iloc[i]

        specz = photozdata_subset['specz_redshift']
        specz_reshape = np.reshape(specz, [1, 1])

        if count_in == 0:
        
            for (columnName, columnData) in photozdata.items():
                
                if columnName == 'specz_name' or columnName == 'coord':
                    
                    hf.create_dataset(columnName, data = np.reshape(np.array(photozdata[columnName]).astype(str), [286401, 1]).astype('S')[i], maxshape = (None, ))
                    
                else:
                    
                    hf.create_dataset(columnName, data = photozdata[columnName][i : i + 1], maxshape = (None, ))
            
            hf.create_dataset('image', data = five_band_image_reshape, chunks = True, maxshape = (None, 5, 127, 127))

        else:
        
            for (columnName, columnData) in photozdata.items():
        
                hf[columnName].resize((hf[columnName].shape[0] + 1), axis = 0)
            
                if columnName == 'specz_name' or columnName == 'coord':
                    
                    hf[columnName][hf[columnName].shape[0] - 1] = np.reshape(np.array(photozdata[columnName]).astype(str), [286401, 1]).astype('S')[i]
                    
                else:
                    
                    hf[columnName][hf[columnName].shape[0] - 1] = photozdata[columnName][i : i + 1]
            
            hf['image'].resize((hf['image'].shape[0] + 1), axis = 0)
            hf['image'][hf['image'].shape[0] - 1, :, :, :] = five_band_image_reshape
        
        count_in = count_in + 1
    
    image_g.close()
    image_r.close()
    image_i.close()
    image_z.close()
    image_y.close()

hf.close()

In [ ]:
# END